In [26]:
import pandas as pd
import random 

In [18]:
nomi_path = r'C:\Users\Lucrezia\Documents\GitHub\CPM_exam_organization\dataset\names_list.csv'
nomi = pd.read_csv(nomi_path, header=None)

nome_cognome = []
for nome in nomi[0]:
    nome_cognome.append(nome.split())

lista_lunghezze = [len(nome) for nome in nome_cognome]
for i,l in enumerate(lista_lunghezze):
    if l != 2:
        lista_nome_cognome = nome_cognome[i]
        lista_rimpicciolita = [lista_nome_cognome[0], lista_nome_cognome[1]+lista_nome_cognome[2]]
        nome_cognome[i] = lista_rimpicciolita

In [19]:
lista_lunghezze = [len(nome) for nome in nome_cognome]
count = 0
for l in lista_lunghezze:
    if l != 2:
        count += 1
print(f'lista_lunghezze ha {count} valori diversi da 2')

lista_lunghezze ha 0 valori diversi da 2


In [31]:
students = pd.DataFrame()
students['id'] = ['st_'+str(i) for i in range(len(nome_cognome))]
students['name'] = [l[0] for l in nome_cognome]
students['surname'] = [l[1] for l in nome_cognome]
courses = ['AFAM_biennio_1', 'AFAM_biennio_2', 'AFAM_triennio_1', 'AFAM_triennio_2', 'AFAM_triennio_3', 'Diploma_1', 'Diploma_2', 'Diploma_3', 'BAC', 'Corsi_di_Base_1', 'Corsi_di_Base_2', 'Corsi_di_Base_3', 'Junior']
random.seed(42)
students['course_of_study'] = random.choices(courses, k=700)
students

,id,name,surname,course_of_study
0,st_0,Simone,Longo,BAC
1,st_1,Andrea,D'Angelo,AFAM_biennio_1
2,st_2,Giuseppe,Fontana,AFAM_triennio_2
3,st_3,Tommaso,Leone,AFAM_triennio_1
4,st_4,Sofia,Esposito,Corsi_di_Base_1
...,...,...,...,...
695,st_695,Giuseppe,Sanna,BAC
696,st_696,Simone,Gallo,BAC
697,st_697,Lorenzo,Moretti,Junior
698,st_698,Sara,Martini,Diploma_1
